<a href="https://colab.research.google.com/github/jameseconnolly/jameseconnolly.github.io/blob/main/CO2Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
# // Opening web sites and web scraping
import requests

# // JSON. This helps us make JSON look prettier and easier to read
import json

# /// Files.  This is part of Collab - allows you to upload and download files
from google.colab import files

# // OS. Sometimes need this for finding working directory
import os

# // For exporting
import pandas as pd
import numpy as np
from pandas import DataFrame as df

In [127]:
# // Scraping a table from iban's website to generate a list of Alpha-3 country codes
df1=pd.read_html('https://www.iban.com/country-codes')
df2=df1[0]
df3 = df2.drop(["Alpha-2 code", "Numeric"], axis=1)
df4 = df3[~df3['Alpha-3 code'].isin(filter_values)]
codes = df4['Alpha-3 code'].tolist()

In [128]:
print(codes)

['AFG', 'ALB', 'DZA', 'ASM', 'AGO', 'ATA', 'ATG', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL', 'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR', 'BFA', 'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CYM', 'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COD', 'COG', 'COK', 'CRI', 'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'DNK', 'DJI', 'DMA', 'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'FLK', 'FRO', 'FJI', 'FIN', 'FRA', 'GUF', 'PYF', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD', 'GLP', 'GUM', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KIR', 'PRK', 'KOR', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MAC', 'MKD', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MTQ', 'MRT', 'MUS', 'MEX', 'FSM', 'MDA', 'MNG', 'MNE', 'MSR', 'MAR', 'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'NLD', 'NCL', 'NZL', 'NIC', 'NER'

In [129]:
url_base = 'http://api.eia.gov/series/?api_key=PLuuh4H0dedLDcMthjNCj0oCjpb85UV1RSUd9u2l&series_id=INTL.4008-8-{}-MMTCD.A'

In [130]:
# // df to be used later
df = pd.DataFrame()

In [131]:
for code in codes:

    # // Build the URL for this iteration of the loop and request HTML:
    URL = url_base.format(code)
    data = requests.get(URL).json()

    for i in data['series']:
        
        # // Adding yearly data in array
        array = np.array(i['data'])

        # // Adding country code in array
        array = np.insert(array,array.shape[1],code,axis=1)

        # // Convert array into df:
        df0 = pd.DataFrame(array)

        # /// Concat each country together
        df = pd.concat([df, df0], ignore_index=True)

In [132]:
df

,0,1,2
0,2021,5.19367574,AFG
1,2020,5.388894815,AFG
2,2019,6.082892279,AFG
3,2018,6.818422849,AFG
4,2017,5.639362838,AFG
...,...,...,...
9182,1984,7.783562489,ZWE
9183,1983,7.584832144,ZWE
9184,1982,7.332897573,ZWE
9185,1981,7.531253431,ZWE


In [133]:
dataframe=df.rename(columns={0:'Year',1:'CO₂ million mTonne',2:'Country'})

In [134]:
# // Import pycountry, to match the country codes to country names, so the data is recognisable in the topojson
import pycountry

In [135]:
def get_country_name(code):
  return pycountry.countries.get(alpha_3=code).name

dataframe['Country Name'] = dataframe['Country'].apply(get_country_name)

In [136]:
dataframe

,Year,CO₂ million mTonne,Country,Country Name
0,2021,5.19367574,AFG,Afghanistan
1,2020,5.388894815,AFG,Afghanistan
2,2019,6.082892279,AFG,Afghanistan
3,2018,6.818422849,AFG,Afghanistan
4,2017,5.639362838,AFG,Afghanistan
...,...,...,...,...
9182,1984,7.783562489,ZWE,Zimbabwe
9183,1983,7.584832144,ZWE,Zimbabwe
9184,1982,7.332897573,ZWE,Zimbabwe
9185,1981,7.531253431,ZWE,Zimbabwe


In [137]:
# // Matching the name of countries in dataframe to those in the topojson
dataframe['Country Name'].replace({"Bahamas":"The Bahamas","Bolivia, Plurinational State of":"Bolivia", "Brunei Darussalam":"Brunei","Congo":"Republic of the Congo","Congo, The Democratic Republic of the":"Democratic Republic of the Congo","Serbia":"Republic of Serbia","North Macedonia":"Macedonia","Guinea-Bissau":"Guinea Bissau","Timor-Leste":"East Timor","Côte d'Ivoire":"Ivory Coast","Iran, Islamic Republic of":"Iran","Moldova, Republic of":"Moldova","Russian Federation":"Russia","Syrian Arab Republic":"Syria","Venezuela, Bolivarian Republic of":"Venezuela","Czechia":"Czech Republic","Dominica":"Dominican Republic","Korea, Democratic People's Republic of":"North Korea","Korea, Republic of":"South Korea","Lao People's Democratic Republic":"Laos","Taiwan, Province of China":"Taiwan","Tanzania, United Republic of":"United Republic of Tanzania","United States":"United States of America","Viet Nam":"Vietnam"}, inplace=True)

In [138]:
dataframe.to_csv('CO2Map.csv', index=False)